# Import Libraries

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
books=pd.read_csv(r"C:\Users\as\OneDrive\Desktop\recommandation\data\Books.csv")
users=pd.read_csv(r"C:\Users\as\OneDrive\Desktop\recommandation\data\Users.csv")
ratings=pd.read_csv(r"C:\Users\as\OneDrive\Desktop\recommandation\data\Ratings.csv")

C:\Users\as\AppData\Local\Temp\ipykernel_10948\4125825301.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books=pd.read_csv(r"C:\Users\as\OneDrive\Desktop\recommandation\data\Books.csv")


In [6]:
books.head(1)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...


In [7]:
users.head(1)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN


In [8]:
ratings.head(1)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0


In [9]:
print(books.shape)
print(users.shape)
print(ratings.shape)

(271360, 8)
(278858, 3)
(1149780, 3)


In [10]:
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [11]:
users.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [12]:
ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [13]:
books.duplicated().sum()

np.int64(0)

In [14]:
users.duplicated().sum()

np.int64(0)

In [15]:
ratings.duplicated().sum()

np.int64(0)

## Popularity Based Recommender System

In [16]:
rating_with_name=ratings.merge(books,on="ISBN")

In [17]:
num_rating_df=rating_with_name.groupby("Book-Title").count()["Book-Rating"].reset_index()
num_rating_df.rename(columns={"Book-Rating":"Num_Rating"},inplace=True)
num_rating_df

,Book-Title,Num_Rating
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241066,Ã?Â?lpiraten.,2
241067,Ã?Â?rger mit Produkt X. Roman.,4
241068,Ã?Â?sterlich leben.,1
241069,Ã?Â?stlich der Berge.,3


In [18]:
avg_rating_df = rating_with_name.groupby('Book-Title')['Book-Rating'].mean().reset_index()
avg_rating_df.rename(columns={'Book-Rating': 'Avg_Rating'}, inplace=True)
avg_rating_df

,Book-Title,Avg_Rating
0,A Light in the Storm: The Civil War Diary of ...,2.250000
1,Always Have Popsicles,0.000000
2,Apple Magic (The Collector's series),0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,0.000000
...,...,...
241066,Ã?Â?lpiraten.,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,5.250000
241068,Ã?Â?sterlich leben.,7.000000
241069,Ã?Â?stlich der Berge.,2.666667


In [19]:
popular_df=num_rating_df.merge(avg_rating_df,on="Book-Title")
popular_df

,Book-Title,Num_Rating,Avg_Rating
0,A Light in the Storm: The Civil War Diary of ...,4,2.250000
1,Always Have Popsicles,1,0.000000
2,Apple Magic (The Collector's series),1,0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
...,...,...,...
241066,Ã?Â?lpiraten.,2,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,4,5.250000
241068,Ã?Â?sterlich leben.,1,7.000000
241069,Ã?Â?stlich der Berge.,3,2.666667


In [20]:
popular_df=popular_df[popular_df["Num_Rating"]>=250].sort_values('Avg_Rating',ascending=False).head(50)
popular_df=popular_df.merge(books,on="Book-Title").drop_duplicates("Book-Title")[['Book-Title','Book-Author','Image-URL-M','Num_Rating','Avg_Rating']]

## Collabrative Based Recommendation System

#### user> 200 book >50

In [21]:
x=rating_with_name.groupby("User-ID").count()["Book-Rating"]>200     
user_with_200rate=x[x].index

In [22]:
filter_rating=rating_with_name[rating_with_name["User-ID"].isin(user_with_200rate)]

In [23]:
y=filter_rating.groupby("Book-Title").count()["Book-Rating"]>=50
famous_book=y[y].index

In [24]:
final_rating=filter_rating[filter_rating["Book-Title"].isin(famous_book)]

In [25]:
pt=final_rating.pivot_table(index="Book-Title",columns="User-ID",values="Book-Rating")

pt.fillna(0,inplace=True)

In [26]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_score=cosine_similarity(pt)

In [27]:
def Recommend(book_name):
    index=np.where(pt.index==book_name)[0][0]
    similar_items=sorted(list(enumerate(similarity_score[index])),key=lambda x:x[1],reverse=True)[1:6]
    data=[]
    for i in similar_items:
        items=[]
        # temp_df=books[books["Book-Title"]==pt.index[i[0]]]
        temp_df = books[books['Book-Title'].str.lower().str.strip() == pt.index[i[0]].strip().lower()]

        items.extend(list(temp_df.drop_duplicates("Book-Title")["Book-Title"].values))
        items.extend(list(temp_df.drop_duplicates("Book-Title")["Book-Author"].values))
        items.extend(list(temp_df.drop_duplicates("Book-Title")["Image-URL-M"].values))
        
        data.append(items)
    return data
        

In [28]:
Recommend("1984")

[['Animal Farm',
  'George Orwell',
  'http://images.amazon.com/images/P/0451526341.01.MZZZZZZZ.jpg'],
 ["The Handmaid's Tale",
  'Margaret Atwood',
  'http://images.amazon.com/images/P/0449212602.01.MZZZZZZZ.jpg'],
 ['Brave New World',
  'Aldous Huxley',
  'http://images.amazon.com/images/P/0060809833.01.MZZZZZZZ.jpg'],
 ['The Vampire Lestat (Vampire Chronicles, Book II)',
  'ANNE RICE',
  'http://images.amazon.com/images/P/0345313860.01.MZZZZZZZ.jpg'],
 ['The Hours : A Novel',
  'Michael Cunningham',
  'http://images.amazon.com/images/P/0312243022.01.MZZZZZZZ.jpg']]

In [29]:
import pickle
pickle.dump(popular_df,open("popular.pkl","wb"))

In [30]:
pickle.dump(pt,open("books_pt.pkl","wb"))
pickle.dump(books,open("books.pkl","wb"))
pickle.dump(similarity_score,open("book_similarity_score.pkl","wb"))